## Documentation

- API Guide can be found in this link https://ib-insync.readthedocs.io/api.html
- Options Example (SPX) with ib_insync https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [48]:
from ib_insync import *
import pandas as pd
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId = 23)

tick = 'AAPL'

stk = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(stk)
ib.reqMarketDataType(4) #wait a bit
[ticker] = ib.reqTickers(stk)
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
ds_chains = pd.DataFrame(chains)
expirations = sorted(ds_chains['expirations'].iloc[0])
strikes = ds_chains['strikes'].iloc[0]
contractID = ds_chains['underlyingConId'].iloc[0]
rights = ['P', 'C'] #easier for algirthm built # P = Put ; C = Call

#Establish Contracts Array
contracts = [Option('AAPL', expiration, strike, right, 'SMART') for right in rights for expiration in expirations for strike in strikes]
contracts = ib.qualifyContracts(*contracts) #select qualified contracts

#organize all Option Contracts into data set
ds_options = pd.DataFrame()
for c in contracts:
    row = {'Symbol': c.symbol, 'Expiration': c.lastTradeDateOrContractMonth , 'Strike': c.strike , 'Right': c.right}
    ds_options = pd.concat([ds_options, pd.DataFrame([row])], axis = 0, ignore_index = True)

ds_options.to_csv('options_02.csv')

#next: call all options during market time to get greek values
ib.disconnect()

Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220708', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220708', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220708', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220708', strike=41.25, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220708', strike=42.5, right='P', exchange='SMART')
Error 200, reqId 12: No security definition has 

In [ ]:
tickers = ib.reqTickers(*contracts)

for t in tickers:
        # dlt = ticker[0].modelGreeks.delta
        row = {'Symbol': t.symbol, 'Expiration': t , 'Strike': stk , 'Right': poc}
        print(ticker[0])
        ds = pd.concat([ds, pd.DataFrame([row])], axis = 0, ignore_index = True)


# tickers = ib.reqTickers(*contracts)


# f = open("output.txt", "a")
# print(contracts[0], file=f)
# f.close()

ib.disconnect()
ds